In [ ]:
import os
import cv2
import json
import glob
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/EECS_442 Final Proj/WLASL

/content/drive/.shortcut-targets-by-id/1Sc7VSl-7PU4L0hxklxePzLVSFNQ7pnLE/EECS_442 Final Proj/WLASL


In [ ]:
!mkdir optical_features

In [ ]:
from torchvision import models as torch_models

# model = torch_models.resnet50(34)
# class ResNet_extractor(torch.nn.Module):
#     def __init__(self, layers=101):
#         super().__init__()
#         if layers == 18:
#             self.resnet = torch_models.resnet18(pretrained=True)
#         elif layers == 34:
#             self.resnet = torch_models.resnet34(pretrained=True)
#         elif layers == 50:
#             self.resnet = torch_models.resnet50(pretrained=True)
#         elif layers == 101:
#             self.resnet = torch_models.resnet101(pretrained=True)
#         else:
#             raise(ValueError('Layers must be 18, 34, 50 or 101.'))

#     def forward(self, x): # batch x 224 x 224 x 3
#         x = self.resnet.conv1(x)
#         x = self.resnet.bn1(x)
#         x = self.resnet.relu(x)
#         x = self.resnet.maxpool(x)

#         x = self.resnet.layer1(x)
#         x = self.resnet.layer2(x)
#         x = self.resnet.layer3(x)
#         x = self.resnet.layer4(x)
#         #print(x.shape)
#         x = self.resnet.avgpool(x)
#         #print(x.shape)
#         x = torch.flatten(x, 1) # batch x 2048
#         return x
model = torch_models.resnet34(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [ ]:
def extract_optical_flow_images(video_path, save_dir, start_frame, desired_frame=50):
  cap = cv2.VideoCapture(video_path)
  cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
  ret, frame1 = cap.read()
  prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
  hsv = np.zeros_like(frame1)
  hsv[..., 1] = 255
  count = 0
  while(count < desired_frame):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    dest = os.path.join(save_dir, str(count).zfill(3)+'.jpg')
    cv2.imwrite(dest, bgr)
    
    # k = cv2.waitKey(30) & 0xff
    # if k == 27:
    #     break
    # elif k == ord('s'):
    #     cv2.imwrite('opticalfb.png', frame2)
    #     cv2.imwrite('opticalhsv.png', bgr)
    prvs = next
    count += 1
  while (count < desired_frame): # pad using last bgr
    dest = os.path.join(save_dir, str(count).zfill(3)+'.jpg')
    cv2.imwrite(dest, bgr)
    count += 1




In [ ]:
from PIL import Image
import math
import torch
def flow_to_feature(vid_id, dir, dest, model):
  img_paths = list(sorted(glob.glob(dir+'*.jpg')))
  images = []
  for path in img_paths:
    img = np.asarray(Image.open(path))
    w, h, c = img.shape
    if w < 226 or h < 226:
      d = 226. - min(w, h)
      sc = 1 + d / min(w, h)
      img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)
    if w > 256 or h > 256:
      img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))
    #print(img.shape)
    images.append(img)
  images = np.asarray(images, dtype=np.float32)
  t, h, w, c = images.shape
  i = int(np.round((h - 224) / 2.))
  j = int(np.round((w - 224) / 2.))
  images = images[:, i:i+224, j:j+224, :]
  print(vid_id)
  assert images.shape == (50, 224, 224, 3)
  features = model(torch.from_numpy(images).permute([0, 3, 1, 2]).cuda()) # frame x 224 x 224 x 3 -> frame x 3 x 224 x 224
  #print(features.shape)
  dest = os.path.join(dest, vid_id+'.npy')
  np.save(dest, features.detach().cpu().numpy())
  del features


In [ ]:
with open('./splits/nslt_100.json', 'r') as f:
  data = json.load(f)

existing = []

existing = list(glob.glob('./videos/*.mp4'))
existing_id = []
for e in existing:
  name = e.split('/')[-1].split('.')[0]
  existing_id.append(name)


In [ ]:
for vid_id in data.keys():
  if vid_id in existing_id:
    flow_to_feature(vid_id, f'./optical_flow_images/{vid_id}/', './optical_features/', model.cuda())

69422
10898
10893
10892
10895
10894
51068
51064
51067
51066
51061
51060
43180
06365
69395
69396
57291
69370
56839
56838
56837
06335
06334
06337
06331
06330
06333
06332
11768
11769
11767
30849
64292
64293
64291
65507
65506
45443
05734
05730
05731
05732
05733
05739
05229
17724
17720
17721
17722
17723
14624
14625
26973
26972
26975
26976
14685
65029
63208
14680
63204
63207
63200
63201
63202
63203
51063
62247
62246
62245
62244
62248
08437
07074
07070
26719
26713
26712
26715
26714
26717
07961
65889
65884
15043
28214
28210
28211
65084
65085
65086
66532
69413
69411
05243
65263
02003
39006
39000
69343
69345
49173
49176
49174
49175
42843
49178
05741
05740
05743
05742
05749
10158
36946
13199
13198
14894
14893
13196
45273
06476
62259
62250
62251
07068
07069
40847
31755
31756
31751
31750
31753
31752
66575
06486
69402
66799
66798
05644
65216
35458
35452
35453
35456
65187
35455
32250
32253
32255
32254
32257
65677
01387
56852
69359
57278
24955
24954
24956
65328
24952
55365
55362
69439
05750
10166
1488

In [ ]:
for vid in data.keys():
  img_folder = os.path.join('./optical_flow_images/', vid)
  

In [ ]:
def extract_optical_flow_cropped(video_path, save_dir, start_frame, desired_frame=50):
  cap = cv2.VideoCapture(video_path)
  cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
  ret, frame1 = cap.read()
  prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
  hsv = np.zeros_like(frame1)
  hsv[..., 1] = 255
  count = 0
  while(count < desired_frame):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    cropped = bgr[137:492, 16:480]
    cropped = cv2.resize(cropped, (224, 224))
    dest = os.path.join(save_dir, str(count).zfill(3)+'.jpg')
    cv2.imwrite(cropped, bgr)
    
    # k = cv2.waitKey(30) & 0xff
    # if k == 27:
    #     break
    # elif k == ord('s'):
    #     cv2.imwrite('opticalfb.png', frame2)
    #     cv2.imwrite('opticalhsv.png', bgr)
    prvs = next
    count += 1
  while (count < desired_frame): # pad using last bgr
    dest = os.path.join(save_dir, str(count).zfill(3)+'.jpg')
    cv2.imwrite(dest, bgr)
    count += 1

In [ ]:
for vid in data.keys():
  if vid in existing_id:
    dest = os.path.join('optical_flow_images', vid)
    vid_path = os.path.join('./videos/', vid+'.mp4')
    if not os.path.exists(dest):
      os.mkdir(dest)
      extract_optical_flow_cropped(vid_path, dest, data[vid]['action'][1])
      break

Cropped